This notebook aims to create rainfall categories - but only for pixels with rainfall as selected e.g. Rainfall_min = 0.1

In [1]:
# Import libraries

import os
import glob
from osgeo import gdal
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import csv
import pprint
import h5py 
import gdal
from gdalconst import *
import datetime

In [2]:
# Set input directory, and change working directory - plug in D drive
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
# select MODIS files
MODISFiles = glob.glob('*H19V08*') 
First_MODISFiles = MODISFiles[:108]
Second_MODISFiles = MODISFiles[108:]
print(len(First_MODISFiles), len(Second_MODISFiles))

108 108


In [4]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [5]:
RainfallFiles = glob.glob('*rainfall*')
First_RainfallFiles = RainfallFiles[:108]
Second_RainfallFiles = RainfallFiles[108:]
print(len(First_MODISFiles), len(Second_MODISFiles))

108 108


In [6]:
number = 0
for i in range(len(RainfallFiles)):
    MODISFile = MODISFiles[number]
    RainfallFile = RainfallFiles[number]
    
    date = MODISFile.split(".A"[1])[2].split(".h")[0]
    year = date[2:4]
    julian_day = date[4:]
    reformatted_dates = datetime.datetime.strptime(year + julian_day, '%y%j').date()
    MODISdate = reformatted_dates.strftime('%y_%m_%d')
    
    Rainfall_date = RainfallFile[2:10]

    print("MODIS: {0} \t Rainfall: {1}".format(MODISdate, Rainfall_date))
    number += 1


MODIS: 02_01_01 	 Rainfall: 02_01_01
MODIS: 02_02_01 	 Rainfall: 02_02_01
MODIS: 02_03_01 	 Rainfall: 02_03_01
MODIS: 02_04_01 	 Rainfall: 02_04_01
MODIS: 02_05_01 	 Rainfall: 02_05_01
MODIS: 02_06_01 	 Rainfall: 02_06_01
MODIS: 02_07_01 	 Rainfall: 02_07_01
MODIS: 02_08_01 	 Rainfall: 02_08_01
MODIS: 02_09_01 	 Rainfall: 02_09_01
MODIS: 02_10_01 	 Rainfall: 02_10_01
MODIS: 02_11_01 	 Rainfall: 02_11_01
MODIS: 02_12_01 	 Rainfall: 02_12_01
MODIS: 03_01_01 	 Rainfall: 03_01_01
MODIS: 03_02_01 	 Rainfall: 03_02_01
MODIS: 03_03_01 	 Rainfall: 03_03_01
MODIS: 03_04_01 	 Rainfall: 03_04_01
MODIS: 03_05_01 	 Rainfall: 03_05_01
MODIS: 03_06_01 	 Rainfall: 03_06_01
MODIS: 03_07_01 	 Rainfall: 03_07_01
MODIS: 03_08_01 	 Rainfall: 03_08_01
MODIS: 03_09_01 	 Rainfall: 03_09_01
MODIS: 03_10_01 	 Rainfall: 03_10_01
MODIS: 03_11_01 	 Rainfall: 03_11_01
MODIS: 03_12_01 	 Rainfall: 03_12_01
MODIS: 04_01_01 	 Rainfall: 04_01_01
MODIS: 04_02_01 	 Rainfall: 04_02_01
MODIS: 04_03_01 	 Rainfall: 04_03_01
M

H19V08

In [14]:
# create BA array for first 9 years (2002 - 2010)
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

first_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in First_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(First_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[(RainfallFile <= 500) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= First_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(First_MODISFiles[NUMBER]) as dataset:
        crs = dataset.read_crs()
    
    file_name = First_MODISFiles[NUMBER]
    file = SD(file_name, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    first_BA[result] += 1
    
    
    NUMBER += 1


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [15]:
# create BA array for first 9 years (2002 - 2010)
# issue im adding one every time theres a fire but resetting it if rainfall is less than .1!
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

second_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in Second_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(Second_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[(RainfallFile <= 500) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= Second_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(filename) as dataset:
        crs = dataset.read_crs()
    
    file = SD(filename, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    second_BA[result] += 1
    
    
    NUMBER += 1


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

In [16]:
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_sub500_H19V08.csv", first_BA, delimiter=",")
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_sub500_H19V08.csv", second_BA, delimiter=",")

In [8]:
# create BA array for first 9 years (2002 - 2010)
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

first_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in First_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(First_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[((RainfallFile >= 500)*(RainfallFile <= 1500)) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= First_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(First_MODISFiles[NUMBER]) as dataset:
        crs = dataset.read_crs()
    
    file_name = First_MODISFiles[NUMBER]
    file = SD(file_name, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    first_BA[result] += 1
    
    NUMBER += 1

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [9]:
# create BA array for first 9 years (2002 - 2010)
# issue im adding one every time theres a fire but resetting it if rainfall is less than .1!
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

second_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in Second_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(Second_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[((RainfallFile >= 500)*(RainfallFile <= 1500)) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= Second_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(filename) as dataset:
        crs = dataset.read_crs()
    
    file = SD(filename, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    second_BA[result] += 1
    
    
    NUMBER += 1


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Use

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\pandas\core\computation\expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Use

In [10]:
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_500-1500_H19V08.csv", first_BA, delimiter=",")
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_500-1500_H19V08.csv", second_BA, delimiter=",")

In [11]:
# create BA array for first 9 years (2002 - 2010)
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

first_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in First_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(First_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[(RainfallFile >= 1500) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= First_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(First_MODISFiles[NUMBER]) as dataset:
        crs = dataset.read_crs()
    
    file_name = First_MODISFiles[NUMBER]
    file = SD(file_name, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    first_BA[result] += 1
    
    
    NUMBER += 1


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [12]:
# create BA array for first 9 years (2002 - 2010)
# issue im adding one every time theres a fire but resetting it if rainfall is less than .1!
inDir = "D:\\masters_data\\MODIS_BA" 
os.chdir(inDir)

second_BA = np.zeros((2400, 2400))
NUMBER = 0

for j in Second_MODISFiles:
    inDir = "D:\\masters_data\\output\\csv\\Rainfall\\H19_V08_arrays"
    os.chdir(inDir)
    RainfallFile = pd.read_csv(Second_RainfallFiles[NUMBER], header =None)
    mask = np.zeros((2400, 2400))
    mask[(RainfallFile >= 1500) == 1] = 1
    
    inDir = "D:\\masters_data\\MODIS_BA" 
    os.chdir(inDir)
    filename= Second_MODISFiles[NUMBER]
    dataset = gdal.Open(filename,GA_ReadOnly)
    subdataset = dataset.GetSubDatasets()[1]
    
    with rio.open(filename) as dataset:
        crs = dataset.read_crs()
    
    file = SD(filename, SDC.READ)   
    
    datasets_dic = file.datasets()
    sds_obj = file.select(0)
    data = sds_obj.get() 
    result = np.where((data > 0) *(mask==1))
    
    second_BA[result] += 1
    
    
    NUMBER += 1


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users

In [13]:
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_above_1500_H19V08.csv", first_BA, delimiter=",")
np.savetxt("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_above_1500_H19V08.csv", second_BA, delimiter=",")